In [4]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
this_stuff=rq.get("https://en.wikipedia.org/wiki/List_of_international_airports_by_country")
new_obj=BeautifulSoup(this_stuff.text)
big_list=[]
parameters=[]
x=0
Region=['Asia','East Asia','Nordic region', 'Americas', 'Africa','United Kingdom','Nordic Region','Eastern Europe', 'Europe','Oceania','Eurasia',"Passenger Roles (2011–2020)",'Caribbean','Central America','Central Asia','South Asia','Southeast Asia','Southwest Asia and the Middle East','Western Europe','Central Europe','[edit]','Southern Europe','North America','South America','[']
list_of_countries={}
for entry in new_obj.find_all('span'):
    if(entry.text=='See also'):
        break
    if(x%2==0 and x>3 and entry.text not in Region):
        list_of_countries[entry.text]=1
    x+=1

In [6]:
m=0
country_l=list(list_of_countries.keys())
big_list=[]
for entry in new_obj.find_all('tr'):
    new_entry = [str(part) for part in entry.text.splitlines() if(part != "")]
    d={}
    if('Location' in new_entry):
        parameters=new_entry
        m+=1
    elif('Location' not in new_entry[0]):
        if(len(new_entry) == len(parameters)):
            d['Country']= country_l[m]
            for x in range(len(parameters)):
                if(parameters[x]== 'IATA' or parameters[x]=='IATA Code'):
                    d['IATA Code']=new_entry[x]
                elif(parameters[x] == '2016 Passengers' or parameters[x] == '2015 Passengers' or parameters[x] == '2014/2015 Passengers'):
                    d['Annual Passengers']= new_entry[x].replace(',','')
                elif(parameters[x] == "2015 Daily Avg."):
                    d['Annual Passengers']= str(int(new_entry[x].replace(',','').replace('>',''))*365)
                else:
                    d[parameters[x]]=new_entry[x]
            big_list.append(d) 

In [26]:
airport_df=pd.DataFrame(big_list)
airport_df.drop(columns=['ICAO','Average','2018 Passengers','Daily Avg.','2014 Passengers','2015 Avg.'],inplace=True)
airport_df=airport_df[(airport_df['Passenger Role'].notna()) | (airport_df['Annual Passengers'].notna()) ]
airport_df.set_index(airport_df['IATA Code'],inplace=True)
airport_df.drop(columns=['IATA Code'],inplace=True)
airport_df.to_csv(path_or_buf='C:/Users/mjhig/Disease Simulator/disease_sim/airports.csv')